In [ ]:
import numpy as np
import pandas as pd
import random
from scipy.sparse import coo_matrix,csc_matrix,csr_matrix,lil_matrix 
import timeit

data = np.load('user_movie_rating.npy')


In [ ]:
print(data.shape)
print(data)
print(data[0:10])

In [ ]:
#read in the first 1 million elements to reduce the size

user_ids = data[:1000000,0]
movie_ids = data[:1000000,1]
ratings = data[:1000000,2]

#and also the full size, but this will be to large to make an array
user_ids_full = data[:,0]
movie_ids_full = data[:,1]
ratings_full = data[:,2]

#how many users selected? 
print("Number of selected users with user_ids is: ", len(user_ids))
print("Number of selected users with user_ids_full is: ", len(user_ids_full))

# creating sparse matrix, only use part of the elements to save memory 
sparse_colMatrix = csc_matrix((ratings, (movie_ids, user_ids)))
sparse_colMatrix_full = csc_matrix((ratings_full, (movie_ids_full, user_ids_full)))
#colMatrix_full = sparse_colMatrix_full.toarray()
#print(sparse_colMatrix)

sparse_rowMatrix = csr_matrix((ratings, (user_ids, movie_ids)))
sparse_rowMatrix_full = csr_matrix((ratings_full, (user_ids_full, movie_ids_full)))
#rowMatrix_full = sparse_rowMatrix_full.toarray()
#print(sparse_rowMatrix)

In [ ]:
rowMatrix = sparse_rowMatrix.toarray()
colMatrix = sparse_colMatrix.toarray()

In [ ]:
np.random.seed(1702)

#This is the example from the lecture slides
example_matrix1 = np.transpose(np.array([[1, 1, 0, 0, 0, 1, 1], 
                          [0, 0, 1, 1, 1, 0, 0],
                          [1, 0, 0, 0, 0, 1, 1],
                          [0, 1, 1, 1, 1, 0, 0]]))
test_perm1 = np.transpose(np.array([[1, 3, 7, 6, 2, 5, 4], 
                                    [4, 2, 1, 3, 6, 7, 5],
                                    [3, 4, 7, 6, 1, 2, 5]]))
test_perm1 = np.subtract(test_perm1, 1) #subtract 1 to start at index 0 like the example in the book

#This is the example from the book chapter 3.3.5
example_matrix2 = np.transpose(np.array([[1, 0, 0, 1, 0], 
                          [0, 0, 1, 0, 0],
                          [0, 1, 0, 1, 1],
                          [1, 0, 1, 1, 0]]))
test_perm2 = np.transpose(np.array([[1, 2, 3, 4, 0], 
                                    [1, 4, 2, 0, 3]]))

def examples(example_matrix, permutation):
    print("Example Matrix:\n", example_matrix)
    print("Hash functions:\n", permutation)

    signatureMatrix = np.full((len(permutation[0]), example_matrix.shape[1]), np.inf)
    print("\nSignature Matrix:\n", signatureMatrix)

    for r in range(0, example_matrix.shape[0]):
        for c in range(0, example_matrix.shape[1]):
            if example_matrix[r][c] == 1:
                for i in range(0, len(permutation[0])):
                    signatureMatrix[i][c] = min(signatureMatrix[i][c], int(permutation[r][i]))

    print("\nSignature Matrix:\n", signatureMatrix)

    
    
examples(example_matrix1, test_perm1)

In [ ]:
np.random.seed(1702)

begin = timeit.default_timer()

def minhashing(data_matrix):
    print("Example Matrix:\n", data_matrix, "with shape:", data_matrix.shape)

    random_perm_matrix = np.zeros((data_matrix.shape[0], 100))

    random_perm = np.arange(data_matrix.shape[0])
    print(random_perm)
    for i in range(100):
        np.random.shuffle(random_perm)
        random_perm_matrix[:, i] = random_perm
    print('The random matrix is\n',random_perm_matrix, "with shape:", random_perm_matrix.shape)

    signatureMatrix = np.full((random_perm_matrix.shape[1], data_matrix.shape[1]), np.inf)
    print("\nSignature Matrix:\n", signatureMatrix, "with shape:", signatureMatrix.shape)
    
    for r in range(0, data_matrix.shape[0]):
        print("Row:", r)
        for c in range(0, data_matrix.shape[1]):
            if data_matrix[r][c] == 1:
                for i in range(0, random_perm_matrix.shape[1]):
                    signatureMatrix[i][c] = min(signatureMatrix[i][c], int(random_perm_matrix[r][i]))
                    

    print("\nSignature Matrix:\n", signatureMatrix)
    
    end = timeit.default_timer() - begin 
    print('The total time it took was ', end/60, ' minutes')
    
    return signatureMatrix

minhashing(colMatrix)


